In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hsbmpy import fraction_bar_plot
from tableanalyser import get_ensg, get_symbol
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

In [ ]:
directory = r'/Volumes/GoogleDrive/My Drive/tesi_magistrale/tesi/gtex/hsbm/oversigma_10tissue'
l=3

In [ ]:
df = pd.read_csv("gene_tf_matrix.txt.gz", sep="\t", low_memory=False).drop(["#.1"], axis=1).drop([0,1], axis=0).set_index("#").drop("GeneSym",axis=1)
df.head()

In [ ]:
df.parallel_apply(lambda x: np.sum(x.astype(float)), axis=0)

In [ ]:
genes = pd.read_csv("gene_symbol.txt", index_col=0)
genes

In [ ]:
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,l))
df_topic_distr = pd.read_csv("%s/topsbm/topsbm_level_%d_word-dist.csv"%(directory,l), index_col=0)

In [ ]:
topic_tf = pd.DataFrame(index=df.columns)
for topic in df_topics.columns:
    print(topic)
    #get P(gene|topic) with symbols
    subdf = df_topic_distr[topic][df_topic_distr[topic]>0]
    subdf.index=[get_symbol(e) for e in subdf.index]
    #get TF for current genes
    cdf = df[df.index.isin(subdf.index)].astype(float)
    #weight TF for probability
    cdf.multiply(subdf, axis=0).fillna(0.)
    #sum # of TF
    c_series = cdf.apply(lambda x: np.sum(x.astype(float)), axis=0)
    topic_tf.insert(0, topic, c_series)

In [ ]:
fraction_tf = topic_tf.fillna(0).transpose()
fraction_tf=fraction_tf.divide(fraction_tf.sum(axis=1), axis=0).fillna(0).to_dict('list')

In [ ]:
fig=plt.figure(figsize=(35,8))
ax=fig.subplots()
x = np.arange(1,1+len(df_topics.columns))
fraction_bar_plot(x, fraction_tf, ax)
#plt.yscale('log')
plt.legend(ncol=20)
plt.show()